"Variance"예제에서 쓰인 sum_reduce() 함수를 예로 들어 암호문 안에 암호 내용을 어떻게 배치하는지에 대한 문제를 말씀 드리고자 합니다.  
(ciphertext packing 이라고 표현합니다)

In [4]:
import numpy as np

1. 동형 암호는 기본 데이터가 2^n 슬롯을 갖는 벡터입니다. 

   수학적 구조상 1개 슬롯 암호문을 계산하는 것과 8192개짜리 암호문을 계산하는 비용이 크게 차이 나지 않기 때문에, throughput 향상을 위해 벡터를 활용합니다. 
2. (역시 수학적인 이유로) 암호문은 벡터지만 indexing은 불가능하고, 대신 벡터 전체를 임의의 칸 만큼 shift할 수 있습니다. 



Variance 계산에 등장하는 sum_reduce() 함수의 중심 아이디어는 다음과 같습니다.  
(이번에는 암호화 하지 않았습니다) 

In [41]:
arr = np.array([1,2,3,4,5,6,7,8])

for i in [1,2,4]:
    tmp = np.copy(arr)
    tmp = np.roll(tmp, i) # 슬롯 밀기
    arr += tmp
    print(i, arr)

1 [ 9  3  5  7  9 11 13 15]
2 [22 18 14 10 14 18 22 26]
4 [36 36 36 36 36 36 36 36]


이렇게 하면 indexing 없이 shift만으로 암호문 전체의 합을 계산할 수 있습니다. 

그러나 암호문 길이가 2^n으로 딱 떨어지지 않으면 sub-optimal한 상황이 벌어집니다.

In [42]:
arr2 = np.array([1,2,3,4,5,6,7,8,9,10])

for i in [1,2,4]:
    tmp = np.copy(arr2)
    tmp = np.roll(tmp, i)
    arr2 += tmp
    print(i, arr2)
print(arr2 == 55, "\n")

arr2 = np.array([1,2,3,4,5,6,7,8,9,10])
for i in [1,2,4,8]:
    tmp = np.copy(arr2)
    tmp = np.roll(tmp, i)
    arr2 += tmp
    print(i, arr2)
print(arr2 == 55)

1 [11  3  5  7  9 11 13 15 17 19]
2 [28 22 16 10 14 18 22 26 30 34]
4 [50 48 46 44 42 40 38 36 44 52]
[False False False False False False False False False False] 

1 [11  3  5  7  9 11 13 15 17 19]
2 [28 22 16 10 14 18 22 26 30 34]
4 [50 48 46 44 42 40 38 36 44 52]
8 [ 96  92  88  84  80  76  82  88  94 100]
[False False False False False False False False False False]


이럴 때는 암호문을 더 길게 한 뒤, 빈칸을 남겨야 합니다. 

In [31]:
arr2 = np.array([1,2,3,4,5,6,7,8,9,10,0,0,0,0,0,0])
for i in [1,2,4,8]:
    tmp = np.copy(arr2)
    tmp = np.roll(tmp, i)
    arr2 += tmp
    print(i, arr2)
print(arr2 == 55)

1 [ 1  3  5  7  9 11 13 15 17 19 10  0  0  0  0  0]
2 [ 1  3  6 10 14 18 22 26 30 34 27 19 10  0  0  0]
4 [11  3  6 10 15 21 28 36 44 52 49 45 40 34 27 19]
8 [55 55 55 55 55 55 55 55 55 55 55 55 55 55 55 55]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


그런데, 내용물보다 암호문이 더 길어지면 정답이 담기는 slot의 위치와 범위가 달라집니다.  

In [44]:
### 1,2,4,8 슬롯만 회전할 경우 ###
arr2 = np.array([1,2,3,4,5,6,7,8,9,10,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
for i in [1,2,4,8]: 
    tmp = np.copy(arr2)
    tmp = np.roll(tmp, i)
    arr2 += tmp

print(i, arr2)

8 [ 1  3  6 10 15 21 28 36 45 55 55 55 55 55 55 55 54 52 49 45 40 34 27 19
 10  0  0  0  0  0  0  0  0]


In [45]:
### 1,2,4,8,16 슬롯 회전할 경우 ###
arr2 = np.array([1,2,3,4,5,6,7,8,9,10,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
for i in [1,2,4,8,16]: 
    tmp = np.copy(arr2)
    tmp = np.roll(tmp, i)
    arr2 += tmp

print(i, arr2)

16 [53 52 51 50 49 48 47 46 45 55 55 55 55 55 55 55 55 55 55 55 55 55 55 55
 55 55 55 55 55 55 55 55 54]


따라서 **암호문의 길이**와 **내용물의 길이**의 관계에 따라 sum()과 같은 낮은 수준의 함수에서도 최적화 방법에 차이가 발생합니다.  

다차원 데이터를 1차원 암호문으로 만든 뒤 다양한 연산을 수행하다보면 이와 유사한 packing 최적화 문제가 지속적으로 발생합니다.  
암호문의 life cycle이 길어지면 사용자가 직접 packing 최적화를 구현하기가 매우 어려워, 장기적으로는 컴파일러 최적화로 해결해야할 문제로 생각됩니다.  

암호문의 기본 특성은
1. slot 개수 (고정)
2. scale (가변)
3. noise budget (가변) 
이 있으며, 

이번 예제의 개념을 확장하면 (최신 연구를 참고하면) 다음과 같은 추가 특성을 생각할 수 있습니다.  
4. valid_slot_mask (시작, 끝, stride)


다양한 경우에서 최적의 packing을 찾는 방법은 앞으로도 많은 연구가 필요할 것으로 생각됩니다. 